In [1]:
import cv2  
import os
os.chdir('/home/samrat/Desktop/SoC-FaceRecog/Age-and-Gender-Recognition-main/models')# The location of models

In [2]:
def detectFace(net,frame,confidence_threshold=0.7):
    frameOpencvDNN=frame.copy()
    #print(frameOpencvDNN.shape)
    frameHeight=frameOpencvDNN.shape[0]
    frameWidth=frameOpencvDNN.shape[1]
    scale=1.0
    size=(227,227)#Size of array required by cnn
    blob=cv2.dnn.blobFromImage(frameOpencvDNN,scale,size,[124.96,115.97,106.13],swapRB=True,crop=False)
    #SwapRB is for changing BGR to RGB
    ######################################Pre-processing Done
    net.setInput(blob)
    detections=net.forward()#Process
    #print(detections)
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>confidence_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDNN,(x1,y1),(x2,y2),(0,255,0),int(round(frameHeight/150)),8)
    return frameOpencvDNN,faceBoxes
        
    


In [3]:
faceProto='opencv_face_detector.pbtxt'
faceModel='opencv_face_detector_uint8.pb'
ageProto='age_deploy.prototxt'
ageModel='age_net.caffemodel'
genderProto='gender_deploy.prototxt'
genderModel='gender_net.caffemodel'

genderList=['Male','Female']
#genderList=['Dumb','Smart']
#ageList=['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']
ageList=[1,5,10,18,27,40,50,80]

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

video=cv2.VideoCapture(0)
padding=20 #CNN
while cv2.waitKey(10)<0:
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break
        
    resultImg,faceBoxes=detectFace(faceNet,frame,0.7)
    
    if not faceBoxes:
        print("No face detected")
    
    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding):min(faceBox[2]+padding, frame.shape[1]-1)]
        blob=cv2.dnn.blobFromImage(face,1.0,(227,227),[124.96,115.97,106.13],swapRB=True,crop=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        
        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=0
        for i in range(8):
            age+=agePreds[0][i]*ageList[i]
            #print(agePreds)
            #print(age)
        age=int(age) 
        print(age)
        cv2.putText(resultImg,f'{gender},{age}',(faceBox[0],faceBox[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2,cv2.LINE_AA)
        cv2.imshow("Detecting age and Gender",resultImg)
        
    #Minor indentation bug in source code
    if cv2.waitKey(33) & 0xFF == 13:#Enter key
        break
            
cv2.destroyAllWindows()

6
13
11
28
21
25
30
13
22
13
10
6
10
3
10
12
16
10
20
11
12
13
10
14
15
22
11
12
23
14
12
11
12
11
12
13
8
10
16
14
18
10
12
15
10
10
10
9
15
11
11
15
11
17
11
14
24
12
16
15
10
20
10
11
11
11
11
14
12
17
20
10
20
17
12
16
14
14
10
14
12
12
10
8
9
32
14
10
20
20
22
25
14
